# Introduction

Through a quick overview/exploration of the existing online datasets, one was selected as the unanimous best. It's well organised, and contains the most data from the most amount of sources.

## Chapter Objective

### Primary 

To organise the dataset with its ingredients as elements of the dataset. 

### Secondary

Preprocess this dataset so it can be used in the following chapters.

# Setup 

In [335]:
#| default_exp recipes.create

In [336]:
#| export
from pyprojroot import here
root = here()
import sys
sys.path.append(str(root))

In [337]:
#| export
import pandas as pd
import numpy as np

import nltk
import spacy
from spacy.matcher import Matcher
from spacy.util import filter_spans

import json
from itertools import groupby
import re
import string
import time

from ast import literal_eval

from tqdm import tqdm
tqdm.pandas()

from food_database.utils.utils import *

from parse_ingredients import parse_ingredient as parse_ingredient_rgx
from ingredient_parser import parse_ingredient as parse_ingredient_nlp

import dask.dataframe as dd
from dask.distributed import Client, LocalCluster
import multiprocessing as mp

from food_database.utils.parallel import *

from pathlib import Path

from sacremoses import MosesTokenizer, MosesDetokenizer
mt, md = MosesTokenizer(lang='en'), MosesDetokenizer(lang='en')

In [338]:
pd.options.mode.chained_assignment = None  # default='warn'

In [339]:
#| export
with open(f'{root}/data/globals/unit_conversions.json') as f:
    unit_list = json.load(f)

# Recipes DF

In [340]:
recipes_df = pd.read_csv('../data/datasets/recipe/recipes_nlg/full_dataset.csv')
recipes_df = recipes_df.drop('Unnamed: 0', axis=1)
recipes_df

,title,ingredients,directions,link,source,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."
...,...,...,...,...,...,...
2231137,Sunny's Fake Crepes,"[""1/2 cup chocolate hazelnut spread (recommend...","[""Spread hazelnut spread on 1 side of each tor...",www.foodnetwork.com/recipes/sunny-anderson/sun...,Recipes1M,"[""chocolate hazelnut spread"", ""tortillas"", ""bu..."
2231138,Devil Eggs,"[""1 dozen eggs"", ""1 paprika"", ""1 salt and pepp...","[""Boil eggs on medium for 30mins."", ""Then cool...",cookpad.com/us/recipes/355411-devil-eggs,Recipes1M,"[""eggs"", ""paprika"", ""salt"", ""choice"", ""miracle..."
2231139,Extremely Easy and Quick - Namul Daikon Salad,"[""150 grams Daikon radish"", ""1 tbsp Sesame oil...","[""Julienne the daikon and squeeze out the exce...",cookpad.com/us/recipes/153324-extremely-easy-a...,Recipes1M,"[""radish"", ""Sesame oil"", ""White sesame seeds"",..."
2231140,Pan-Roasted Pork Chops With Apple Fritters,"[""1 cup apple cider"", ""6 tablespoons sugar"", ""...","[""In a large bowl, mix the apple cider with 4 ...",cooking.nytimes.com/recipes/1015164,Recipes1M,"[""apple cider"", ""sugar"", ""kosher salt"", ""bay l..."


In [341]:
recipes_df = recipes_df.astype({'title': 'string', 'link': 'string', 'source': 'string'})

In [342]:
recipes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2231142 entries, 0 to 2231141
Data columns (total 6 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   title        string
 1   ingredients  object
 2   directions   object
 3   link         string
 4   source       string
 5   NER          object
dtypes: object(3), string(3)
memory usage: 102.1+ MB


## Initial Exploration

A brief exploration to understand the dataset.

### Sources

In [343]:
recipes_df['link'].head()

0     www.cookbooks.com/Recipe-Details.aspx?id=44874
1    www.cookbooks.com/Recipe-Details.aspx?id=699419
2     www.cookbooks.com/Recipe-Details.aspx?id=10570
3    www.cookbooks.com/Recipe-Details.aspx?id=897570
4    www.cookbooks.com/Recipe-Details.aspx?id=659239
Name: link, dtype: string

In [344]:
def get_base_url(url):
    base_url = re.search(r'^.+?[^\/:](?=[?\/]|$)', url).group(0)
    base_url = base_url.replace('www.', '')
    return base_url

get_base_url(recipes_df['link'].iloc[0])

'cookbooks.com'

In [345]:
recipes_df['link'].apply(get_base_url).value_counts()

link
cookbooks.com          896341
food.com               499616
epicurious.com         129444
tastykitchen.com        78768
myrecipes.com           64895
allrecipes.com          64602
cookpad.com             61020
cookeatshare.com        59307
yummly.com              51963
tasteofhome.com         51594
foodnetwork.com         49443
food52.com              48501
kraftrecipes.com        42010
recipeland.com          24418
recipes-plus.com        20524
cooking.nytimes.com     16367
foodandwine.com         15436
seriouseats.com         12632
foodgeeks.com            8963
cookstr.com              8797
online-cookbook.com      5691
chowhound.com            5671
vegetariantimes.com      4578
delish.com               3880
landolakes.com           2492
foodrepublic.com         2259
lovefood.com             1930
Name: count, dtype: int64

## Sampling DF

A large dataset, we want a small sample to be able to work with quickly during exploration.

In [346]:
recipes_df_full = recipes_df.copy()
recipes_df = recipes_df_full.sample(300, ignore_index=False, random_state=777)
recipes_df.shape[0], recipes_df_full.shape[0]/recipes_df.shape[0]

(300, 7437.14)

In [347]:
# adding tracked recipes
recipes_df = pd.concat([recipes_df, recipes_df_full.loc[[2006319, 931097]]], axis=0)

In [348]:
recipes_df.to_feather('../data/local/recipe/partial/recipe/0.feather')

# Ingredients DF

Since we will be working with each individial ingredient, we want to explode each ingredient as its own row, which we could tie to its recipe through a MultiIndex.

In [349]:
# read_csv reads lists as strings -> convering them to lists beforehand
recipes_df['ingredients'] = recipes_df['ingredients'].apply(lambda x: literal_eval(x))

In [350]:
ingredients_df = recipes_df.explode('ingredients')
ingredients_df

,title,ingredients,directions,link,source,NER
1746116,Turtle Thumbprints,"2/3 cup Land O Lakes Butter, softened","[""Combine butter, sugar, egg yolks and vanilla...",www.landolakes.com/recipe/1086/turtle-thumbprints,Recipes1M,"[""Butter"", ""sugar"", ""Eggs"", ""vanilla"", ""flour""..."
1746116,Turtle Thumbprints,1/2 cup sugar,"[""Combine butter, sugar, egg yolks and vanilla...",www.landolakes.com/recipe/1086/turtle-thumbprints,Recipes1M,"[""Butter"", ""sugar"", ""Eggs"", ""vanilla"", ""flour""..."
1746116,Turtle Thumbprints,2 Land O Lakes Eggs (yolks only),"[""Combine butter, sugar, egg yolks and vanilla...",www.landolakes.com/recipe/1086/turtle-thumbprints,Recipes1M,"[""Butter"", ""sugar"", ""Eggs"", ""vanilla"", ""flour""..."
1746116,Turtle Thumbprints,1 teaspoon vanilla,"[""Combine butter, sugar, egg yolks and vanilla...",www.landolakes.com/recipe/1086/turtle-thumbprints,Recipes1M,"[""Butter"", ""sugar"", ""Eggs"", ""vanilla"", ""flour""..."
1746116,Turtle Thumbprints,1 1/2 cups all-purpose flour,"[""Combine butter, sugar, egg yolks and vanilla...",www.landolakes.com/recipe/1086/turtle-thumbprints,Recipes1M,"[""Butter"", ""sugar"", ""Eggs"", ""vanilla"", ""flour""..."
...,...,...,...,...,...,...
931097,Pineapple Spiced Chicken And Rice,1/4 cup chopped red bell pepper,"[""Drain pineapple; reserve 1 tablespoon juice....",www.allrecipes.com/recipe/232611/pineapple-spi...,Gathered,"[""Pineapple"", ""garlic"", ""ground thyme"", ""groun..."
931097,Pineapple Spiced Chicken And Rice,1 cup Jasmine rice,"[""Drain pineapple; reserve 1 tablespoon juice....",www.allrecipes.com/recipe/232611/pineapple-spi...,Gathered,"[""Pineapple"", ""garlic"", ""ground thyme"", ""groun..."
931097,Pineapple Spiced Chicken And Rice,1 1/2 cups reduced-sodium chicken broth,"[""Drain pineapple; reserve 1 tablespoon juice....",www.allrecipes.com/recipe/232611/pineapple-spi...,Gathered,"[""Pineapple"", ""garlic"", ""ground thyme"", ""groun..."
931097,Pineapple Spiced Chicken And Rice,1 tablespoon chopped fresh basil,"[""Drain pineapple; reserve 1 tablespoon juice....",www.allrecipes.com/recipe/232611/pineapple-spi...,Gathered,"[""Pineapple"", ""garlic"", ""ground thyme"", ""groun..."


In [351]:
# setting indices for the ingredients in each recipe (multiindex)
ingredients_df['ingredient'] = ingredients_df.groupby(ingredients_df.index).cumcount()
ingredients_df = ingredients_df.set_index([ingredients_df.index, 'ingredient'])
ingredients_df.index = ingredients_df.index.set_names(['recipe', 'ingredient'])
ingredients_df.head()

title                            ingredients  \
recipe  ingredient                                                              
1746116 0           Turtle Thumbprints  2/3 cup Land O Lakes Butter, softened   
        1           Turtle Thumbprints                          1/2 cup sugar   
        2           Turtle Thumbprints       2 Land O Lakes Eggs (yolks only)   
        3           Turtle Thumbprints                     1 teaspoon vanilla   
        4           Turtle Thumbprints           1 1/2 cups all-purpose flour   

                                                           directions  \
recipe  ingredient                                                      
1746116 0           ["Combine butter, sugar, egg yolks and vanilla...   
        1           ["Combine butter, sugar, egg yolks and vanilla...   
        2           ["Combine butter, sugar, egg yolks and vanilla...   
        3           ["Combine butter, sugar, egg yolks and vanilla...   
        4           ["Combine butter, sugar, egg yolks and vanilla...   

                                                                 link  \
recipe  ingredient                                                      
1746116 0           www.landolakes.com/recipe/1086/turtle-thumbprints   
        1           www.landolakes.com/recipe/1086/turtle-thumbprints   
        2           www.landolakes.com/recipe/1086/turtle-thumbprints   
        3           www.landolakes.com/recipe/1086/turtle-thumbprints   
        4           www.landolakes.com/recipe/1086/turtle-thumbprints   

                       source  \
recipe  ingredient              
1746116 0           Recipes1M   
        1           Recipes1M   
        2           Recipes1M   
        3           Recipes1M   
        4           Recipes1M   

                                                                  NER  
recipe  ingredient                                                     
1746116 0           ["Butter", "sugar", "Eggs", "vanilla", "flour"...  
        1           ["Butter", "sugar", "Eggs", "vanilla", "flour"...  
        2           ["Butter", "sugar", "Eggs", "vanilla", "flour"...  
        3           ["Butter", "sugar", "Eggs", "vanilla", "flour"...  
        4           ["Butter", "sugar", "Eggs", "vanilla", "flour"...

In [352]:
# selecting columns
ingredients_df = ingredients_df.drop(columns=['title', 'directions', 'link', 'source'])
ingredients_df.rename(columns={'ingredients':'ingredient_string'}, inplace=True)
ingredients_df

ingredient_string  \
recipe  ingredient                                            
1746116 0             2/3 cup Land O Lakes Butter, softened   
        1                                     1/2 cup sugar   
        2                  2 Land O Lakes Eggs (yolks only)   
        3                                1 teaspoon vanilla   
        4                      1 1/2 cups all-purpose flour   
...                                                     ...   
931097  10                  1/4 cup chopped red bell pepper   
        11                               1 cup Jasmine rice   
        12          1 1/2 cups reduced-sodium chicken broth   
        13                 1 tablespoon chopped fresh basil   
        14                  Salt and ground pepper to taste   

                                                                  NER  
recipe  ingredient                                                     
1746116 0           ["Butter", "sugar", "Eggs", "vanilla", "flour"...  
        1           ["Butter", "sugar", "Eggs", "vanilla", "flour"...  
        2           ["Butter", "sugar", "Eggs", "vanilla", "flour"...  
        3           ["Butter", "sugar", "Eggs", "vanilla", "flour"...  
        4           ["Butter", "sugar", "Eggs", "vanilla", "flour"...  
...                                                               ...  
931097  10          ["Pineapple", "garlic", "ground thyme", "groun...  
        11          ["Pineapple", "garlic", "ground thyme", "groun...  
        12          ["Pineapple", "garlic", "ground thyme", "groun...  
        13          ["Pineapple", "garlic", "ground thyme", "groun...  
        14          ["Pineapple", "garlic", "ground thyme", "groun...  

[2511 rows x 2 columns]

# Processing (1: Initial)

Now that we have the ingredients dataframe, we want to process it so it for use in the following chapters.

This section will involve structuring the ingredient string's important important information to be easily accessed when required. ie. from an ingredient string we will have fields for its name, quantity, and unit. 

## Object Types

In [353]:
ingredients_df = ingredients_df.astype({'ingredient_string': 'string'})
ingredients_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2511 entries, (1746116, 0) to (931097, 14)
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ingredient_string  2511 non-null   string
 1   NER                2511 non-null   object
dtypes: object(1), string(1)
memory usage: 58.4+ KB


## Parsing Information

### Name, Quantity, Unit Parsing

We need to extract two things from the ingredient strings: 

1. Name
2. Quantity

These are written in the form of a string, so to parse this information we need to use NLP, specficially Named Entity Recognition (NER). 


Here we could train our own model to do precisely what we want, however this would require a bit of work. We'd need to manually create example parses from a dataset, and figure out how training a model in SpaCy works (~5hr work).

Let's first try the existing libraries to see what we can gain from it.

### Research


- https://github.com/Glorf/recipenlg (datasets thesis explains how they extracted their ingredients in the 'NER' column)
- https://deepgram.com/learn/turning-recipes-into-data-with-named-entity-recognition (good example on training SpaCy)
- https://ingredient-parser.readthedocs.io/en/latest/index.html (python SpaCy library)
- https://archive.nytimes.com/open.blogs.nytimes.com/2015/04/09/extracting-structured-data-from-recipes-using-conditional-random-fields/ (2015 article how NyTimes' recipe search functionality was created)
    - https://github.com/mtlynch/ingredient-phrase-tagger (open source code extracted from it)

These methods all use an NLP method called Named Entity Recognition (NER).

We could train our own NER model, but first lets take a look at the existing models to see if they're fit for our purposes.

In [354]:
recipe = ingredients_df.loc[ingredients_df.first_valid_index()[0]]['ingredient_string']
ingredient = recipe.loc[0]
ingredient, recipe

('2/3 cup Land O Lakes Butter, softened',
 ingredient
 0              2/3 cup Land O Lakes Butter, softened
 1                                      1/2 cup sugar
 2                   2 Land O Lakes Eggs (yolks only)
 3                                 1 teaspoon vanilla
 4                       1 1/2 cups all-purpose flour
 5                             20 caramels, unwrapped
 6    2 tablespoons Land O Lakes Heavy Whipping Cream
 7                                    48 pecan halves
 8            1/2 cup real semi-sweet chocolate chips
 9                             2 teaspoons shortening
 Name: ingredient_string, dtype: string)

### Existing Libraries

#### RegEx

RegEx would be the simplest method of doing this, however I'm expecting it not to be able to deal with more complex strings of text.

In [355]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/steph/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [356]:
parsed_ingredient_rgx = parse_ingredient_rgx(ingredient)
parsed_ingredient_rgx

Ingredient(name='Land O Lakes Butter', quantity=2.5, unit='cup', comment='softened', original_string='2/3 cup Land O Lakes Butter, softened')

In [357]:
[parse_ingredient_rgx(ingredient) for ingredient in recipe]

[Ingredient(name='Land O Lakes Butter', quantity=2.5, unit='cup', comment='softened', original_string='2/3 cup Land O Lakes Butter, softened'),
 Ingredient(name='sugar', quantity=1.5, unit='cup', comment='', original_string='1/2 cup sugar'),
 Ingredient(name='Land O Lakes Eggs', quantity=2.0, unit='', comment='(yolks only)', original_string='2 Land O Lakes Eggs (yolks only)'),
 Ingredient(name='vanilla', quantity=1.0, unit='tsp', comment='', original_string='1 teaspoon vanilla'),
 Ingredient(name='all-purpose flour', quantity=1.5, unit='cup', comment='', original_string='1 1/2 cups all-purpose flour'),
 Ingredient(name='caramels', quantity=20.0, unit='', comment='unwrapped', original_string='20 caramels'),
 Ingredient(name='Land O Lakes Heavy Whipping Cream', quantity=2.0, unit='tbsp', comment='', original_string='2 tablespoons Land O Lakes Heavy Whipping Cream'),
 Ingredient(name='pecan halves', quantity=48.0, unit='', comment='', original_string='48 pecan halves'),
 Ingredient(name='

It's pretty good, but even in these six ingredients we can spot errors. It can't convert fraction numbers (2/3 : 2.5).

#### NLP

This is a [pretrained SpaCy NER model](https://ingredient-parser.readthedocs.io/en/latest/index.html) for the exact problemset. Let's see how this performs:

In [358]:
parsed_ingredient_nlp = parse_ingredient_nlp(ingredient)
parsed_ingredient_nlp

ParsedIngredient(name=IngredientText(text='Land O Lakes Butter', confidence=0.989295), amount=[IngredientAmount(quantity='0.667', unit='cups', confidence=0.99965, APPROXIMATE=False, SINGULAR=False)], preparation=IngredientText(text='softened', confidence=0.989225), comment=None, other=None, sentence='2/3 cup Land O Lakes Butter, softened')

In [359]:
[parse_ingredient_nlp(ingredient) for ingredient in recipe]

[ParsedIngredient(name=IngredientText(text='Land O Lakes Butter', confidence=0.989295), amount=[IngredientAmount(quantity='0.667', unit='cups', confidence=0.99965, APPROXIMATE=False, SINGULAR=False)], preparation=IngredientText(text='softened', confidence=0.989225), comment=None, other=None, sentence='2/3 cup Land O Lakes Butter, softened'),
 ParsedIngredient(name=IngredientText(text='sugar', confidence=0.991945), amount=[IngredientAmount(quantity='0.5', unit='cups', confidence=0.998894, APPROXIMATE=False, SINGULAR=False)], preparation=None, comment=None, other=None, sentence='1/2 cup sugar'),
 ParsedIngredient(name=IngredientText(text='Land O Lakes Eggs', confidence=0.962857), amount=[IngredientAmount(quantity='2', unit='', confidence=0.994399, APPROXIMATE=False, SINGULAR=False)], preparation=None, comment=IngredientText(text='(yolks only)', confidence=0.984288), other=None, sentence='2 Land O Lakes Eggs (yolks only)'),
 ParsedIngredient(name=IngredientText(text='vanilla', confidence=

### Chosen Parse Method

I think using NLP sounds much better, but it's has a few flaws right off the bat. Most noteworthy is it doesn't recognise shortened unit for cups (ie. c. != cup). 

We could simply fix this in the preprocessing.

### Preprocessing

A little bit of preprocessing is required here.

#### Cup (c.) Units

In [360]:
#| export 
def preprocess_cup_units(ingredient_string):
    ingredient_string = re.sub(r'\bc\.', 'cup', ingredient_string)
    return ingredient_string

In [361]:
assert preprocess_cup_units('5 c. water') == '5 cup water'

#### OR string

This was confusing the ingredients in later steps.

Here we want to remove the second option of ingredients.

In [362]:
ingredients_df['ingredient_string'][ingredients_df['ingredient_string'].str.contains(' or ')].loc[1936429, 5]

'3 to 4 tablespoons palm sugar, thinly sliced, or dark brown sugar'

##### Tagging Ingedient String

For this we it will be helpful to be able to get a pos_tag (noun, verb, etc) of the strings. This involved tokenizing, and detokenize the text. 

Lets test how much this will mutate the data.

In [363]:
tokenized = ingredients_df['ingredient_string'].apply(lambda x: detokenize(mt.tokenize(x)))

In [364]:
transformation_diffs = tokenized[tokenized != ingredients_df['ingredient_string']]
transformation_diffs.shape[0], ingredients_df.shape[0]

(30, 2511)

In [365]:
pd.set_option('display.max_colwidth', None)

In [366]:
transformation_diffs.to_frame('ingredient_string').join(ingredients_df['ingredient_string'], lsuffix='_transformed', rsuffix='_original')

ingredient_string_transformed  \
recipe  ingredient                                                                                      
954817  2                                      2/3 cup low-carb sugar substitute (such as Swerve (R))   
1984757 1                                                         1 1/2 teaspoons fresh sage leaves *   
        6                                                 1 1/2 teaspoons chopped fresh sage leaves *   
1396331 2                   1 Large rutabaga, or several smaller ones, peeled and cut into 3/4 "cubes   
1369744 10                                            1 slice rustic white bread, torn into 1 "pieces   
        18                                         1-2 slices rustic white bread, torn into 1 "pieces   
1613023 8                                              1 teaspoon golpar (angelica powder) - optional   
324733  0                                                        1 1/2 c. warm water (110 ~ to 115 ~)   
928740  4                                  2 tablespoons coffee-flavored liqueur (such as Kahlua (R))   
1930042 13                                                        2 apples, peeled, cored and chopped   
953258  4                                    2 tablespoons brown sugar substitute (e.g., Splenda (R))   
1505154 8                                                         1/4 cup warm water (110 ° to 115 °)   
840045  2                                                             1 c. M & M's, Skittles, Reese's   
1462944 11                                                   12 (6-inch) fajita-size corn tortillas *   
1379076 1                                                       2 tablespoons unsalted butter, melted   
        2                                                             2 tablespoons fresh lemon juice   
        3                                                                         2 tablespoons sugar   
        4           1 vanilla bean, halved lengthwise and scraped, or 1 teaspoon pure vanilla extract   
        5                                 2-3 tablespoons Peach Flavored liquor or Dark Rum or Brandy   
        6                           4 firm, ripe peaches (1 to 1 1/2 pounds total), halved and pitted   
        8                                                                1 cup chilled whipping cream   
        9                                                         8oz (1 container) mascarpone cheese   
        10                                               2 tablespoons syrup from the roasted peaches   
        11                                                                 1 teaspoon vanilla extract   
        13                                                        All-purpose flour, for work surface   
        14                                                  2 sheets puff pastry (17.3-ounce package)   
        15                                                       1egg + 2 teaspoons water for eggwash   
2158575 2                                                             1 x Salt pork -- about 1/2 "wid   
1199867 5                                                                  black pepper (read * NOTE)   
931097  0                                                  1 (20 ounce) can DOLE (R) Pineapple Slices   

                                                                            ingredient_string_original  
recipe  ingredient                                                                                      
954817  2                                        2/3 cup low-carb sugar substitute (such as Swerve(R))  
1984757 1                                                           1 1/2 teaspoons fresh sage leaves*  
        6                                                   1 1/2 teaspoons chopped fresh sage leaves*  
1396331 2                    1 Large rutabaga, or several smaller ones, peeled and cut into 3/4" cubes  
1369744 10                                             1 slice rustic white bread

-> This seems good, most remain unchanged, whilst the others are changed only with slight differences of the position of some punctiation.

In [367]:
pd.reset_option('display.max_colwidth')

In [368]:
#| export
tag_map = {'NN': 'noun', 'VB': 'verb', 'JJ': 'adj', 'RB': 'adv'}
def tag_ingredient_string(ingredient_string, is_item=True):
    tagged_words = nltk.pos_tag(nltk.word_tokenize(ingredient_string))
    mapped_words = [{'word': w[0], 'tag': tag_map[w[1][:2]] if w[1][:2] in tag_map else 'nan'} for w in tagged_words]
    if is_item and mapped_words:
        mapped_words[-1]['tag'] = 'noun' # making sure last word is noun, as these are items that we are labelling
    return mapped_words

In [369]:
tag_ingredient_string('green chilli pepper')

[{'word': 'green', 'tag': 'adj'},
 {'word': 'chilli', 'tag': 'noun'},
 {'word': 'pepper', 'tag': 'noun'}]

In [370]:
assert tag_ingredient_string('green chilli pepper') == [{'word': 'green', 'tag': 'adj'},
                                                        {'word': 'chilli', 'tag': 'noun'},
                                                        {'word': 'pepper', 'tag': 'noun'}]

In [371]:
#| export
def preprocess_remove_ors(ingredient_string):
    
    or_search = re.search(r'\bor\b', ingredient_string)
    if not or_search: return ingredient_string

    prev_string = ingredient_string[:or_search.span()[0]]
    following_string = ingredient_string[or_search.span()[-1]:]

    prev_tokens = tag_ingredient_string(prev_string, is_item=False)
    following_tokens = tag_ingredient_string(following_string, is_item=False)

    prev_nouns = any([token for token in prev_tokens if token['tag'] == 'noun'])

    if re.search(r'\(\s?or.*\)', ingredient_string):
        ingredient_string = re.sub(r'\(\s?or.*\)', '', ingredient_string)
    elif re.search(r'(\bor\b).*\d', ingredient_string) and prev_nouns:
        ingredient_string = re.sub(r'(\bor\b).*', '', ingredient_string)
    else:
        if not following_tokens: 
            ingredient_string = prev_string
        else:
            i = 0
            while i < len(following_tokens) and not following_tokens[i]['tag'] == 'noun': i += 1
            if i < len(following_tokens) and following_tokens[i]['tag'] == 'noun':
                if i < len(following_tokens)-1: pass
                if i == 0:
                    following_tokens = []
                else:
                    following_tokens = following_tokens[i:]
            else:
                following_tokens = []
            ingredient_words = prev_string.split(' ') + [word['word'] for word in following_tokens]
            ingredient_string = detokenize(sorted(set(ingredient_words), key=ingredient_words.index))
    
    # postprocess
    ingredient_string = ingredient_string.strip()
    ingredient_string = re.sub(r'\s\s+', ' ', ingredient_string) # double spaces
    ingredient_string = re.sub(r'[,\(.]$', '', ingredient_string) # trailing punctuation

    return ingredient_string

In [372]:
tests = {
    '4 egg whites or 1/2 c. Egg Beaters': '4 egg whites',
    'olive or sunflower oil': 'olive oil',
    'olive oil or sunflower oil': 'olive oil',
    '1/4 teaspoon salt or to taste': '1/4 teaspoon salt',
    '3 sticks butter or margarine': '3 sticks butter',
    '1 or 2 onions, chopped': '1 onions, chopped',
    '4 whole Medium (or 6 Small) Peaches, Peeled And Sliced': '4 whole Medium Peaches, Peeled And Sliced',
}

for ingredient_string, desired_string in tests.items():
    transformed = preprocess_remove_ors(ingredient_string)
    assert transformed == desired_string

In [373]:
pd.set_option('display.max_colwidth', None)

In [374]:
transform_or_removed = ingredients_df['ingredient_string'].apply(preprocess_remove_ors)
transform_or_removed[transform_or_removed != ingredients_df['ingredient_string']].to_frame('ingredient_string') \
    .join(ingredients_df['ingredient_string'], lsuffix='_transformed', rsuffix='_original')

ingredient_string_transformed  \
recipe  ingredient                                                                                            
1828339 0                                                      2 (225 g) cans grape leaves, drained (canned   
1703    0                                                                          1 pkg. applesauce, spice   
231060  4                                                                                  1/2 tsp. vanilla   
2195357 0                                                                         4 x Cutlets blue-eyed cod   
        3                                                                               1/2 tsp Dry oregano   
...                                                                                                     ...   
1936429 2           2 to 4 fresh red Holland chiles such as Fresno or cayenne, stemmed and coarsely chopped   
        3                                                                                     1 piece fresh   
        5                                         3 to 4 tablespoons palm sugar, thinly sliced, brown sugar   
        10                                                                                   20 green beans   
        13                 1 cup (scant ounce/30 grams) loosely packed fresh lemon basil, Thai basil leaves   

                                                                                                                                                         ingredient_string_original  
recipe  ingredient                                                                                                                                                                   
1828339 0                                                                                                                  2 (225 g) cans grape leaves, drained (canned or bottled)  
1703    0                                                                                                                               1 pkg. applesauce, spice or carrot cake mix  
231060  4                                                                                                                                         1/2 tsp. vanilla or rum flavoring  
2195357 0                                                                                                            4 x Cutlets blue-eyed cod or possibly snapper cutlets, up to 6  
        3                                                                                                                                    1/2 tsp Dry oregano or possibly rigini  
...                                                                                                                                                                             ...  
1936429 2                                                   2 to 4 fresh red Holland chiles or other fresh long, red chiles such as Fresno or cayenne, stemmed and coarsely chopped  
        3           1 piece fresh or thawed, frozen turmeric, 1 1/2 inches (4 centimeters) long, peeled and coarsely chopped (about 1 1/2 teaspoons), or 1 teaspoon ground turmeric  
        5                                                                                                         3 to 4 tablespoons palm sugar, thinly sliced, or dark brown sugar  
        10                                                   20 green beans or 10 long beans, stemmed and cut into 1/2 inch (12 millimeter) pieces (about 1 cup/6 ounces/170 grams)  
        13                                                                     1 cup (scant 1 ounce/30 grams) loosely packed fresh lemon basil, Thai basil, or Italian basil leaves  

[117 rows x 2 columns]

In [375]:
pd.reset_option('display.max_colwidth')

#### Applying Preprocessing

In [376]:
#| export 
def ner_preprocess_ingredient_string(ingredient_string):
    ingredient_string = preprocess_cup_units(ingredient_string)
    ingredient_string = preprocess_remove_ors(ingredient_string)
    return ingredient_string

In [377]:
transform_whole = ingredients_df['ingredient_string'].apply(ner_preprocess_ingredient_string)
transform_whole[transform_whole != ingredients_df['ingredient_string']].to_frame('ingredient_string') \
    .join(ingredients_df['ingredient_string'], lsuffix='_transformed', rsuffix='_original')

ingredient_string_transformed  \
recipe  ingredient                                                 
1828339 0           2 (225 g) cans grape leaves, drained (canned   
1703    0                               1 pkg. applesauce, spice   
        2                                          1/4 cup water   
        4                                  1 cup nuts (optional)   
894410  0                                            2 cup water   
...                                                          ...   
599284  2                                       1 cup sour cream   
        3                            1 cup grated Cheddar cheese   
1357213 0                           2 cup drained sweet potatoes   
        1                                             1 cup milk   
        4                                          1/2 cup sugar   

                                           ingredient_string_original  
recipe  ingredient                                                     
1828339 0           2 (225 g) cans grape leaves, drained (canned o...  
1703    0                 1 pkg. applesauce, spice or carrot cake mix  
        2                                                1/4 c. water  
        4                                        1 c. nuts (optional)  
894410  0                                                  2 c. water  
...                                                               ...  
599284  2                                             1 c. sour cream  
        3                                  1 c. grated Cheddar cheese  
1357213 0                                 2 c. drained sweet potatoes  
        1                                                   1 c. milk  
        4                                                1/2 c. sugar  

[429 rows x 2 columns]

In [378]:
ingredients_df['ingredient_string'] = transform_whole

### Applying Parsing

In [379]:
#| export
def parse_ingredient_string(ingredient_string):
    nlp_parse = parse_ingredient_nlp(ingredient_string)
    parsed_ingredient = {
        'name': nlp_parse.name.text if nlp_parse.name else None,
        'quantity': nlp_parse.amount[0].quantity if nlp_parse.amount else None,
        'unit': nlp_parse.amount[0].unit if nlp_parse.amount else None,
        'comment': nlp_parse.comment.text if nlp_parse.comment else None,
        'preparation': nlp_parse.preparation.text if nlp_parse.preparation else None
    }
    return parsed_ingredient

In [380]:
parse_ingredient_string(ingredient)

{'name': 'Land O Lakes Butter',
 'quantity': '0.667',
 'unit': 'cups',
 'comment': None,
 'preparation': 'softened'}

In [381]:
ingredients_df['parsed'] = ingredients_df['ingredient_string'].apply(parse_ingredient_string)
ingredients_df['parsed']

recipe   ingredient
1746116  0             {'name': 'Land O Lakes Butter', 'quantity': '0...
         1             {'name': 'sugar', 'quantity': '0.5', 'unit': '...
         2             {'name': 'Land O Lakes Eggs', 'quantity': '2',...
         3             {'name': 'vanilla', 'quantity': '1', 'unit': '...
         4             {'name': 'all-purpose flour', 'quantity': '1.5...
                                             ...                        
931097   10            {'name': 'red bell pepper', 'quantity': '0.25'...
         11            {'name': 'Jasmine rice', 'quantity': '1', 'uni...
         12            {'name': 'reduced-sodium chicken broth', 'quan...
         13            {'name': 'fresh basil', 'quantity': '1', 'unit...
         14            {'name': 'Salt and ground pepper', 'quantity':...
Name: parsed, Length: 2511, dtype: object

### Expanding into Columns

In [382]:
expanded = pd.json_normalize(ingredients_df['parsed'])
expanded.index = ingredients_df.index
expanded

name quantity        unit  \
recipe  ingredient                                                      
1746116 0                    Land O Lakes Butter    0.667        cups   
        1                                  sugar      0.5        cups   
        2                      Land O Lakes Eggs        2               
        3                                vanilla        1    teaspoon   
        4                      all-purpose flour      1.5        cups   
...                                          ...      ...         ...   
931097  10                       red bell pepper     0.25        cups   
        11                          Jasmine rice        1         cup   
        12          reduced-sodium chicken broth      1.5        cups   
        13                           fresh basil        1  tablespoon   
        14                Salt and ground pepper     None        None   

                         comment preparation  
recipe  ingredient                            
1746116 0                   None    softened  
        1                   None        None  
        2           (yolks only)        None  
        3                   None        None  
        4                   None        None  
...                          ...         ...  
931097  10                  None     chopped  
        11                  None        None  
        12                  None        None  
        13                  None     chopped  
        14              to taste        None  

[2511 rows x 5 columns]

In [383]:
expanded.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2511 entries, (1746116, 0) to (931097, 14)
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         2494 non-null   object
 1   quantity     2346 non-null   object
 2   unit         2346 non-null   object
 3   comment      297 non-null    object
 4   preparation  675 non-null    object
dtypes: object(5)
memory usage: 181.8+ KB


In [384]:
expanded = expanded.astype('string')

In [385]:
ingredients_df = pd.concat([ingredients_df, expanded], axis=1)
ingredients_df.drop(['parsed'], axis=1, inplace=True, errors='ignore')
ingredients_df

ingredient_string  \
recipe  ingredient                                            
1746116 0             2/3 cup Land O Lakes Butter, softened   
        1                                     1/2 cup sugar   
        2                  2 Land O Lakes Eggs (yolks only)   
        3                                1 teaspoon vanilla   
        4                      1 1/2 cups all-purpose flour   
...                                                     ...   
931097  10                  1/4 cup chopped red bell pepper   
        11                               1 cup Jasmine rice   
        12          1 1/2 cups reduced-sodium chicken broth   
        13                 1 tablespoon chopped fresh basil   
        14                  Salt and ground pepper to taste   

                                                                  NER  \
recipe  ingredient                                                      
1746116 0           ["Butter", "sugar", "Eggs", "vanilla", "flour"...   
        1           ["Butter", "sugar", "Eggs", "vanilla", "flour"...   
        2           ["Butter", "sugar", "Eggs", "vanilla", "flour"...   
        3           ["Butter", "sugar", "Eggs", "vanilla", "flour"...   
        4           ["Butter", "sugar", "Eggs", "vanilla", "flour"...   
...                                                               ...   
931097  10          ["Pineapple", "garlic", "ground thyme", "groun...   
        11          ["Pineapple", "garlic", "ground thyme", "groun...   
        12          ["Pineapple", "garlic", "ground thyme", "groun...   
        13          ["Pineapple", "garlic", "ground thyme", "groun...   
        14          ["Pineapple", "garlic", "ground thyme", "groun...   

                                            name quantity        unit  \
recipe  ingredient                                                      
1746116 0                    Land O Lakes Butter    0.667        cups   
        1                                  sugar      0.5        cups   
        2                      Land O Lakes Eggs        2               
        3                                vanilla        1    teaspoon   
        4                      all-purpose flour      1.5        cups   
...                                          ...      ...         ...   
931097  10                       red bell pepper     0.25        cups   
        11                          Jasmine rice        1         cup   
        12          reduced-sodium chicken broth      1.5        cups   
        13                           fresh basil        1  tablespoon   
        14                Salt and ground pepper     <NA>        <NA>   

                         comment preparation  
recipe  ingredient                            
1746116 0                   <NA>    softened  
        1                   <NA>        <NA>  
        2           (yolks only)        <NA>  
        3                   <NA>        <NA>  
        4                   <NA>        <NA>  
...                          ...         ...  
931097  10                  <NA>     chopped  
        11                  <NA>        <NA>  
        12                  <NA>        <NA>  
        13                  <NA>     chopped  
        14              to taste        <NA>  

[2511 rows x 7 columns]

In [386]:
ingredients_df = ingredients_df.fillna(pd.NA)

In [387]:
#| export
def pipeline_parse_ingredient_string_parsing(ingredients_df):

    ingredients_df['parsed'] = parallel_apply(
        ingredients_df['full_string'], 
        parse_ingredient_string, 
        meta=pd.Series(dtype='object'), 
        npartitions=500
    )
    
    ingredients_df['parsed'] = ingredients_df['parsed'].apply(literal_eval)

    expanded = pd.json_normalize(ingredients_df['parsed'])
    expanded = expanded.astype('string')
    expanded.set_index(ingredients_df.index, inplace=True)
    ingredients_df = pd.concat([ingredients_df, expanded], axis=1)

    ingredients_df.drop(['parsed'], axis=1, inplace=True, errors='ignore')

    return ingredients_df

### Evaluating Results

In [388]:
ingredients_df[ingredients_df['unit'].isnull()].drop('NER', axis=1).head(30)

ingredient_string  \
recipe  ingredient                                                      
1828339 1                                                        salt   
824709  0                                                       bread   
        1                                             ground cinnamon   
        2                                                       sugar   
        3                                           squeeze margarine   
855127  1                                              tomato, sliced   
        2                                             Parmesan cheese   
        3                                                   olive oil   
        4                   Italian spices (rosemary, thyme, oregano)   
767911  8                                          parsley to garnish   
        9                                          pimento to garnish   
1042455 6           crystallized ginger, minced for garnish (optio...   
2058895 6                                             salt and pepper   
2196831 5                        Salt and freshly ground black pepper   
1653851 9                   kosher salt & freshly ground black pepper   
939793  11                                              salt to taste   
        12                               ground black pepper to taste   
1956519 5                   green olives, for garnish olives, chopped   
1546037 0                                      FOR THE PUMPKIN PUREE:   
        2                                          FOR THE PIE CRUST:   
        9                                            FOR THE FILLING:   
1314074 4                          Kosher salt, freshly ground pepper   
820354  7                                                chopped nuts   
745912  1                                  Lawry's taco seasoning mix   
1591514 5                                            candied cherries   
274238  3                                             pepper and salt   
1606545 4                            dressing Sesame ginger, to taste   
        5                                        cilantro for garnish   
373388  3                                              vanilla wafers   
        4                                   pie filling (your choice)   

                                                         name quantity  unit  \
recipe  ingredient                                                             
1828339 1                                                salt     <NA>  <NA>   
824709  0                                               bread     <NA>  <NA>   
        1                                     ground cinnamon     <NA>  <NA>   
        2                                               sugar     <NA>  <NA>   
        3                                   squeeze margarine     <NA>  <NA>   
855127  1                                              tomato     <NA>  <NA>   
        2                                     Parmesan cheese     <NA>  <NA>   
        3                                           olive oil     <NA>  <NA>   
        4                                      Italian spices     <NA>  <NA>   
767911  8                                             parsley     <NA>  <NA>   
        9                                             pimento     <NA>  <NA>   
1042455 6                                 crystallized ginger     <NA>  <NA>   
2058895 6                                     salt and pepper     <NA>  <NA>   
2196831 5                Salt and freshly ground black pepper     <NA>  <NA>   
1653851 9           kosher salt & freshly ground black pepper     <NA>  <NA>   
939793  11                                               salt     <NA>  <NA>   
        12                                ground black pepper     <NA>  <NA>   
1956519 5                                        green olives     <NA>  <NA>   
1546037 0                                                <NA>     <NA>  <NA>   
        2                          

In [389]:
ingredients_df[ingredients_df['unit'].isnull() & ingredients_df['quantity'].notnull()].drop('NER', axis=1).head(30)

,,ingredient_string,name,quantity,unit,comment,preparation
recipe,ingredient,,,,,,


In [390]:
ingredients_df[ingredients_df['quantity'] == 0]

,,ingredient_string,NER,name,quantity,unit,comment,preparation
recipe,ingredient,,,,,,,


In [391]:
ingredients_df.drop('NER', axis=1).sample(30)

,,ingredient_string,name,quantity,unit,comment,preparation
recipe,ingredient,,,,,,
1097688,1,"1 cup butter, room temperature (do not substit...",butter,1,cup,(do not substitute),room temperature
1600752,12,"1 avocado peeled, pitted, and diced",avocado,1,,<NA>,"peeled, pitted, and diced"
92858,6,"2 eggs, well beaten",eggs,2,,<NA>,well beaten
537007,3,"1 large onion, chopped",onion,1,large,<NA>,chopped
2158575,9,2 Tbsp. Cilantro -- minced,Cilantro,2,Tbsps,<NA>,minced
1379076,3,2 tablespoons sugar,sugar,2,tablespoons,<NA>,<NA>
147316,2,2/3 cup peanut butter,peanut butter,0.667,cups,<NA>,<NA>
947940,5,1/4 cup white sugar,white sugar,0.25,cups,<NA>,<NA>
303741,2,1 can cream of chicken soup,chicken soup,1,can,cream of,<NA>


#### NAs

In [392]:
ingredients_df[ingredients_df['name'].isna()]

ingredient_string  \
recipe  ingredient                                                      
1956519 0           1 (1 1/4 ounce) envelope Lipton Recipe Secrets...   
1546037 0                                      FOR THE PUMPKIN PUREE:   
        2                                          FOR THE PIE CRUST:   
        9                                            FOR THE FILLING:   
        16                            15 ounces, weight Pumpkin Puree   
1806301 5                          1 tbsp. sliced pimiento (optional)   
1396644 0               For the Vanilla Bean and Green Grape Chicken:   
        11                        For the Vanilla-Scented Rice Pilaf:   
1378004 8                     2 tsp, plus 1 TBL kosher salt - divided   
879616  7                                   1 Tbsp. gluten (optional)   
1379076 0                             For the Vanilla-Roasted Peaches   
        7                                   For the Mascarpone Cream:   
        12                                         For the Napoleons:   
        15                       1egg + 2 teaspoons water for eggwash   
2076879 2                                    1 as required Katakuriko   
1956528 0                     4 x swordfish steaks - (abt 6 ounce ea)   
1269565 0                                                    Cookies:   

                                                                  NER  name  \
recipe  ingredient                                                            
1956519 0           ["Recipe Secrets savory", "sour cream", "ricot...  <NA>   
1546037 0           ["Pumpkin", "Flour", "Salt", "Butter", "Egg", ...  <NA>   
        2           ["Pumpkin", "Flour", "Salt", "Butter", "Egg", ...  <NA>   
        9           ["Pumpkin", "Flour", "Salt", "Butter", "Egg", ...  <NA>   
        16          ["Pumpkin", "Flour", "Salt", "Butter", "Egg", ...  <NA>   
1806301 5           ["biscuit mix", "poultry seasoning", "sour cre...  <NA>   
1396644 0           ["Vanilla", "olive oil", "butter", "chicken br...  <NA>   
        11          ["Vanilla", "olive oil", "butter", "chicken br...  <NA>   
1378004 8           ["bacon", "bone", "sausage", "yellow onion", "...  <NA>   
879616  7           ["water", "bread flour", "sugar", "salt", "bak...  <NA>   
1379076 0           ["Peaches", "unsalted butter", "lemon juice", ...  <NA>   
        7           ["Peaches", "unsalted butter", "lemon juice", ...  <NA>   
        12          ["Peaches", "unsalted butter", "lemon juice", ...  <NA>   
        15          ["Peaches", "unsalted butter", "lemon juice", ...  <NA>   
2076879 2           ["tenderloin", "Salt", "Vegetable oil", "clove...  <NA>   
1956528 0           ["swordfish steaks -", "salt", "lemon juice Di...  <NA>   
1269565 0           ["Cookies", "cake mix", "solid pack pumpkin", ...  <NA>   

                   quantity      unit  \
recipe  ingredient                      
1956519 0                 1  envelope   
1546037 0              <NA>      <NA>   
        2              <NA>      <NA>   
        9              <NA>      <NA>   
        16               15    ounces   
1806301 5                 1      tbsp   
1396644 0              <NA>      <NA>   
        11             <NA>      <NA>   
1378004 8                 2      tsps   
879616  7                 1      Tbsp   
1379076 0              <NA>      <NA>   
        7              <NA>      <NA>   
        12             <NA>      <NA>   
        15                1             
2076879 2                 1             
1956528 0              <NA>      <NA>   
1269565 0              <NA>      <NA>   

                                                              comment  \
recipe  ingredient                                                      
1956519 0           Lipton Recipe Secrets savory herb with garlic ...   
1546037 0                                     FOR THE PUMPKIN PUREE :   
        2                                         FOR THE PIE CRUST :   
        9        

The parsing does a good job of exclusing strings which aren't ingredients. It does miss out a few obscure ingredients (glutne, pimiento, swordfish steaks). 

Most of the entries here should be removed, apart from these obscure ingrdients, which are obscure, and the minority. Here we can use the parsing to remove ingredients.

In [393]:
ingredients_df = ingredients_df[~ingredients_df['name'].isna()]

### Postprocessing

In [394]:
ingredients_df = ingredients_df.astype({
    'ingredient_string': 'string'
})
ingredients_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2494 entries, (1746116, 0) to (931097, 14)
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ingredient_string  2494 non-null   string
 1   NER                2494 non-null   object
 2   name               2494 non-null   string
 3   quantity           2339 non-null   string
 4   unit               2339 non-null   string
 5   comment            280 non-null    string
 6   preparation        672 non-null    string
dtypes: object(1), string(6)
memory usage: 155.6+ KB


#### Non-Numeric Quantities

In [395]:
#| export
def is_number(s):
    for split in s.split('.'):
        if not split.isdigit():
            return False
    return True

In [396]:
_ = ingredients_df[(ingredients_df['quantity'].notnull())]
non_numeric_quantities = _[~(_['quantity'].apply(is_number))]['quantity']
non_numeric_quantities

recipe   ingredient
2195357  0                   4x
         4                   1x
1570340  5               1-0.25
606258   9                     
1857320  7                 8-12
2034999  4                2-0.5
1442064  3                  3-6
1916172  1                1-0.5
1396331  0                  3-4
1806301  10                 Few
1369744  9                0.5-1
         18                 1-2
1600752  14                    
1342486  20                 4-5
1613023  1                  3-4
         3                  1-2
123994   4                     
686704   0                 8-10
1330524  0                  3-4
118024   0                1-1.5
286364   4                     
1320064  7                0.5-1
305911   0                  6-8
808085   4                  5-6
         5                  3-5
402239   7                     
1307725  3                     
1408069  3                  4-5
480244   0                  4-5
         1                 9-12
         3          

In [397]:
#| export
def clean_quantity(quantity):
    
    if pd.isnull(quantity) or quantity == '': return quantity

    quantity = quantity.lower()
    quantity = quantity.replace('x', '')
    quantity = re.sub(r'\.$', '', quantity)

    if "-" in quantity:
        splits = quantity.split("-")
        if all([is_number(n) for n in splits]):
            quantity = sum([float(n) for n in splits])/len(splits)
    if quantity in ['several', 'few']: 
        quantity = 3.0
    if quantity == 'half': 
        quantity = 0.5

    try:
        quantity = float(quantity)
    except ValueError:
        quantity = pd.NA

    return quantity

In [398]:
clean_quantity('0.5')

0.5

In [399]:
non_numeric_quantities = non_numeric_quantities.to_frame('original')
non_numeric_quantities['cleaned'] = non_numeric_quantities['original'].apply(clean_quantity)
non_numeric_quantities

original cleaned
recipe  ingredient                  
2195357 0                 4x     4.0
        4                 1x     1.0
1570340 5             1-0.25   0.625
606258  9                           
1857320 7               8-12    10.0
2034999 4              2-0.5    1.25
1442064 3                3-6     4.5
1916172 1              1-0.5    0.75
1396331 0                3-4     3.5
1806301 10               Few     3.0
1369744 9              0.5-1    0.75
        18               1-2     1.5
1600752 14                          
1342486 20               4-5     4.5
1613023 1                3-4     3.5
        3                1-2     1.5
123994  4                           
686704  0               8-10     9.0
1330524 0                3-4     3.5
118024  0              1-1.5    1.25
286364  4                           
1320064 7              0.5-1    0.75
305911  0                6-8     7.0
808085  4                5-6     5.5
        5                3-5     4.0
402239  7                           
1307725 3                           
1408069 3                4-5     4.5
480244  0                4-5     4.5
        1               9-12    10.5
        3            several     3.0
1505154 0              1-0.5    0.75
        5              1-0.5    0.75
58987   0                3-4     3.5
704090  5           0.5-0.75   0.625
565606  5                2-3     2.5
8981    6                6-7     6.5
394397  3                6-8     7.0
619199  0              10-12    11.0
1723278 9                           
1719483 8                 1x     1.0
        11              half     0.5
529533  4                           
1267559 11               few     3.0
1379076 5                2-3     2.5
2158575 1                 1x     1.0
        2                 1x     1.0
        3                 1x     1.0
        5                 2x     2.0
        7                 1x     1.0
        8                 4x     4.0
2005640 1             1-0.75   0.875
        7             1-0.25   0.625
        9            1-0.333  0.6665
1777734 2            1-0.333  0.6665
96825   0              1-1.5    1.25
1936429 2                2-4     3.0
        5                3-4     3.5
1401651 4

In [400]:
cleaned_quantity = ingredients_df['quantity'].apply(clean_quantity).astype('string')
cleaned_quantity

recipe   ingredient
1746116  0             0.667
         1               0.5
         2               2.0
         3               1.0
         4               1.5
                       ...  
931097   10             0.25
         11              1.0
         12              1.5
         13              1.0
         14             <NA>
Name: quantity, Length: 2494, dtype: string

In [401]:
assert ingredients_df[cleaned_quantity == '0'].empty 
assert ingredients_df[cleaned_quantity.str.contains('-')].empty 

In [402]:
ingredients_df['quantity'] = cleaned_quantity

#### RecipesDF NER Field

The author's of the dataset actually already performed a form of parsing in the ingredients using an NLP method Named Entity Recognition (NER). They trained their own model using their own manual labels ([info](https://www.researchgate.net/publication/345308878_Cooking_recipes_generator_utilizing_a_deep_learning-based_language_model)). 

Let's compare this to our own NLP's results.

In [403]:
#| export
def find_ner_match(ingredient):

    ner_match = pd.NA

    ner_ingredients = ingredient['NER'] if isinstance(ingredient['NER'], np.ndarray) else literal_eval(ingredient['NER'])
    ingredient_index = ingredient.name[1] if isinstance(ingredient.name, tuple) else ingredient['ingredient'] # parallel non-multi index

    if ingredient_index in ner_ingredients:
        search_ingredient = str(ner_ingredients[ingredient_index])
        if search_ingredient in ingredient['name']:
            ner_match = search_ingredient
        elif ingredient['name'] in search_ingredient:
            ner_match = ingredient['name']
    if not pd.notnull(ner_match): # the dataset's NER cuts out some of the ingredients, making the indices not match. In this case a search through the whole NER array is required.
        for search_ingredient in ner_ingredients:
            search_ingredient = str(search_ingredient)
            if search_ingredient in ingredient['name']:
                ner_match = search_ingredient
            elif ingredient['name'] in search_ingredient:
                ner_match = ingredient['name']
            if pd.notnull(ner_match): break
        
    return ner_match

In [404]:
ingredients_df.iloc[0]

ingredient_string                2/3 cup Land O Lakes Butter, softened
NER                  ["Butter", "sugar", "Eggs", "vanilla", "flour"...
name                                               Land O Lakes Butter
quantity                                                         0.667
unit                                                              cups
comment                                                           <NA>
preparation                                                   softened
Name: (1746116, 0), dtype: object

In [405]:
find_ner_match(ingredients_df.iloc[0])

'Butter'

In [406]:
ingredients_df['name.ner'] = ingredients_df[['NER', 'name']].apply(find_ner_match, axis=1).astype('string')

In [407]:
ingredients_df[ingredients_df['name.ner'].isnull()].shape[0] / ingredients_df.shape[0]

0.03648757016840417

5% of the ingredients don't match the dataset's NER extraction. Taking a look to see why:

In [408]:
pd.set_option('display.max_colwidth', None)
ingredients_df[ingredients_df['name.ner'].isnull()].join(recipes_df[['directions', 'ingredients']], on='recipe')[['name', 'directions', 'ingredients', 'NER']].head(10)

,,name,directions,ingredients,NER
recipe,ingredient,,,,
1828339,6,dried apricots,"[""Unroll the vine leaves carefully and put them into a bowl of water to remove the preserving liquid."", ""Pat them dry on absorbent kitchen paper towel."", ""You will need about 30 leaves; reserve the remainder."", ""Bring a large saucepan of salted water to boil and stir in the turmeric."", ""Add the rice and simmer for 30 minutes or until the rice is just tender."", ""Drain thoroughly and turn into a bowl."", ""Heat the oil in a small frying pan."", ""Add the onion to the rice with the apricots, sultanas, spices, mint and lemon juice."", ""Mix well together."", ""Take one vine leaf at a time."", ""Place it flat on the work surface and put a heaped teaspoon of the rice mixture in the centre."", ""Fold the base of the leaf over the filling, then fold over the first one side, then the other."", ""Fold over the top to make a neat parcel."", ""Continue making parcels until you have used up all the filling."", ""Line a large frying pan with leftover vine leaves."", ""Arrange the parcels, seam sides down, in a single layer in the pan."", ""Cover the layer with more leaves, then make a second layer of parcels and cover them with leaves."", ""Pour on the orange juice and water and cover the pan."", ""Cook gently over low heat for 1 hour, adding a little boiling water from time to time, if necessary."", ""To serve warm, allow the parcels to cool slightly in the pan, then arrange uncooked vine leaves on a flat serving dish and carefully transfer the parcels to the dish."", ""Serve immediately."", ""To serve cold, allow the parcels to cool completely in the pan, then arrange on top of uncooked vine leaves on a serving dish as above."", ""Cover and chill.""]","[2 (225 g) cans grape leaves, drained (canned or bottled), salt, 12 teaspoon ground turmeric, 175 g long grain brown rice, 1 tablespoon vegetable oil, 1 small onion, peeled and finely chopped, 100 g dried apricots, finely chopped, 50 g sultanas, 1 pinch ground cinnamon, 1 pinch ground allspice, 1 tablespoon chopped of fresh mint, 1 teaspoon lemon juice, 300 ml unsweetened orange juice, 150 ml water]","[""grape leaves"", ""salt"", ""ground turmeric"", ""long grain brown rice"", ""vegetable oil"", ""onion"", ""sultanas"", ""ground cinnamon"", ""ground allspice"", ""mint"", ""lemon juice"", ""orange juice"", ""water""]"
1703,0,pkg. applesauce,"[""Combine all ingredients."", ""Mix well."", ""Bake at 350\u00b0 for 60 to 65 minutes in a tube pan that has been greased and floured.""]","[1 pkg. applesauce, spice or carrot cake mix, 4 eggs, 1/4 c. water, 1 jar mincemeat, 1 c. nuts (optional)]","[""carrot cake"", ""eggs"", ""water"", ""mincemeat"", ""nuts""]"
317211,5,container Cool Whip,"[""With fork, mix together flour, 1/2 cup pecans and butter. Press into 9 x 13-inch pan."", ""Bake for 15 minutes at 350\u00b0."", ""Cool completely.""]","[1 1/4 c. flour, 1/2 c. pecans, finely chopped, 1/2 c. softened butter, 1 (8 oz.) pkg. cream cheese, softened, 1/2 c. powdered sugar, 1 (8 oz.) container Cool Whip, 2 pkg. pistachio instant pudding mix, 3 c. milk, 3/4 c. chopped pecans]","[""flour"", ""pecans"", ""butter"", ""cream cheese"", ""powdered sugar"", ""pistachio instant pudding"", ""milk"", ""pecans""]"
2195357,0,blue-eyed cod,"[""Put 4 to 6 blue-eyed cod or possibly snapper cutlets into a shallow dish and add in extra virgin olive oil, lemon juice, dry oregano or possibly rigini and a smashed clove garlic."", ""Add in a few thin yellow strips of lemon peel then cover and chill for at least one hour, turning once or possibly twice when you think of it."", ""Make sure the fire on the barbecue is well burned down then cook the steaks for about 3 min each side."", ""Baste often during cooking with the marinade."", ""Finish with a good squeeze of fresh lemon juice and a tiny sprinkle of oregano or possibly some minced fresh parsley."", ""Serve immediately."", ""If it rains, cook inside in a frying pan or possibly on

In [409]:
pd.reset_option('display.max_colwidth')

We could use this NER as a sort of third check/filter as we need to simplify the ingredients in order to match them with their density. Why don't we just cut out the recipes where NER's don't match up and see what we're left with. 

In [410]:
len(ingredients_df.index.unique(level=0))

302

In [411]:
(len(ingredients_df.index.unique(level=0)) - len(ingredients_df[ingredients_df['name.ner'].isnull()].index.unique(level=0)) )/ len(ingredients_df.index.unique(level=0))

0.7649006622516556

This cuts out 1/4 of the dataset. That's a lot, but we do have a lot of data to work with. If we can gain something from this parsing it might be worth it. 

What this might help with is filtering obscure ingredients that woudln't be matched with the food databases in the future chapters. Let's see how many unique ingredients we have with those cut out.

In [412]:
ingredients_df_filtered = ingredients_df[~ingredients_df.index.isin(ingredients_df[ingredients_df['name.ner'].isnull()].index.unique(level=0), level=0)]
len(ingredients_df_filtered.index.unique(level=0))

231

In [413]:
ingredients_df_filtered['name'].nunique() / ingredients_df['name'].nunique()

0.733567046450482

-> This proportionally removes the number of unique ingredients, so it's not doing anything significant in reducing obscurity.

What it can serve as is a way of filtering out the main ingredient from the name. It seems to be more harsh at eliminating unnecessary words than our method. 

In [414]:
ingredients_df

ingredient_string  \
recipe  ingredient                                            
1746116 0             2/3 cup Land O Lakes Butter, softened   
        1                                     1/2 cup sugar   
        2                  2 Land O Lakes Eggs (yolks only)   
        3                                1 teaspoon vanilla   
        4                      1 1/2 cups all-purpose flour   
...                                                     ...   
931097  10                  1/4 cup chopped red bell pepper   
        11                               1 cup Jasmine rice   
        12          1 1/2 cups reduced-sodium chicken broth   
        13                 1 tablespoon chopped fresh basil   
        14                  Salt and ground pepper to taste   

                                                                  NER  \
recipe  ingredient                                                      
1746116 0           ["Butter", "sugar", "Eggs", "vanilla", "flour"...   
        1           ["Butter", "sugar", "Eggs", "vanilla", "flour"...   
        2           ["Butter", "sugar", "Eggs", "vanilla", "flour"...   
        3           ["Butter", "sugar", "Eggs", "vanilla", "flour"...   
        4           ["Butter", "sugar", "Eggs", "vanilla", "flour"...   
...                                                               ...   
931097  10          ["Pineapple", "garlic", "ground thyme", "groun...   
        11          ["Pineapple", "garlic", "ground thyme", "groun...   
        12          ["Pineapple", "garlic", "ground thyme", "groun...   
        13          ["Pineapple", "garlic", "ground thyme", "groun...   
        14          ["Pineapple", "garlic", "ground thyme", "groun...   

                                            name quantity        unit  \
recipe  ingredient                                                      
1746116 0                    Land O Lakes Butter    0.667        cups   
        1                                  sugar      0.5        cups   
        2                      Land O Lakes Eggs      2.0               
        3                                vanilla      1.0    teaspoon   
        4                      all-purpose flour      1.5        cups   
...                                          ...      ...         ...   
931097  10                       red bell pepper     0.25        cups   
        11                          Jasmine rice      1.0         cup   
        12          reduced-sodium chicken broth      1.5        cups   
        13                           fresh basil      1.0  tablespoon   
        14                Salt and ground pepper     <NA>        <NA>   

                         comment preparation         name.ner  
recipe  ingredient                                             
1746116 0                   <NA>    softened           Butter  
        1                   <NA>        <NA>            sugar  
        2           (yolks only)        <NA>             Eggs  
        3                   <NA>        <NA>          vanilla  
        4                   <NA>        <NA>            flour  
...                          ...         ...              ...  
931097  10                  <NA>     chopped  red bell pepper  
        11                  <NA>        <NA>     Jasmine rice  
        12                  <NA>        <NA>          chicken  
        13                  <NA>     chopped      fresh basil  
        14              to taste        <NA>             Salt  

[2494 rows x 8 columns]

In [415]:
ingredients_df = ingredients_df.rename({'name.ner': 'name.name', 'name': 'name.description', 'ingredient_string': 'ingredient_string'}, axis=1)
ingredients_df = ingredients_df[['name.name', 'name.description', 'quantity', 'unit', 'comment', 'preparation', 'ingredient_string']]
ingredients_df.head()

name.name     name.description quantity      unit  \
recipe  ingredient                                                     
1746116 0             Butter  Land O Lakes Butter    0.667      cups   
        1              sugar                sugar      0.5      cups   
        2               Eggs    Land O Lakes Eggs      2.0             
        3            vanilla              vanilla      1.0  teaspoon   
        4              flour    all-purpose flour      1.5      cups   

                         comment preparation  \
recipe  ingredient                             
1746116 0                   <NA>    softened   
        1                   <NA>        <NA>   
        2           (yolks only)        <NA>   
        3                   <NA>        <NA>   
        4                   <NA>        <NA>   

                                        ingredient_string  
recipe  ingredient                                         
1746116 0           2/3 cup Land O Lakes Butter, softened  
        1                                   1/2 cup sugar  
        2                2 Land O Lakes Eggs (yolks only)  
        3                              1 teaspoon vanilla  
        4                    1 1/2 cups all-purpose flour

Decision: Opting to keep the ingredients not identified by NER instead of fitlering them.

In [416]:
ingredients_df[ingredients_df['name.name'].isnull()]

name.name                name.description quantity  \
recipe  ingredient                                                      
1828339 6               <NA>                  dried apricots    100.0   
1703    0               <NA>                 pkg. applesauce      1.0   
317211  5               <NA>             container Cool Whip      1.0   
2195357 0               <NA>                   blue-eyed cod      4.0   
895702  5               <NA>             container Cool Whip      1.0   
...                      ...                             ...      ...   
1777734 3               <NA>  tub, COOL WHIP Whipped Topping      1.0   
2177242 1               <NA>                  5 spice powder      1.0   
        3               <NA>                        scallion      3.0   
1097688 0               <NA>                          BATTER     <NA>   
1401651 4               <NA>                   ras el hanout            

                              unit          comment     preparation  \
recipe  ingredient                                                    
1828339 6                        g             <NA>  finely chopped   
1703    0                                     spice            <NA>   
317211  5                                      <NA>            <NA>   
2195357 0                  Cutlets             <NA>            <NA>   
895702  5                                      <NA>            <NA>   
...                            ...              ...             ...   
1777734 3                                      <NA>          thawed   
2177242 1                 teaspoon             <NA>            <NA>   
        3                    heads  white part only  finely chopped   
1097688 0                     <NA>             <NA>            <NA>   
1401651 4           generous pinch             <NA>            <NA>   

                                                   ingredient_string  
recipe  ingredient                                                    
1828339 6                       100 g dried apricots, finely chopped  
1703    0                                   1 pkg. applesauce, spice  
317211  5                              1 (8 oz.) container Cool Whip  
2195357 0                                  4 x Cutlets blue-eyed cod  
895702  5                             1 (12 oz.) container Cool Whip  
...                                                              ...  
1777734 3           1 tub (12 oz.) COOL WHIP Whipped Topping, thawed  
2177242 1                               1 teaspoon five-spice powder  
        3           3 scallion heads white part only, finely chopped  
1097688 0                                                     BATTER  
1401651 4                             generous pinches ras el hanout  

[91 rows x 7 columns]

In [417]:
ingredients_df['name.name'] = ingredients_df.apply(lambda ingredient: ingredient['name.name'] if pd.notnull(ingredient['name.name']) else ingredient['name.description'], axis=1)

## Cleaning Null Values

Making null values consistent.

In [418]:
ingredients_df

name.name              name.description quantity  \
recipe  ingredient                                                           
1746116 0                    Butter           Land O Lakes Butter    0.667   
        1                     sugar                         sugar      0.5   
        2                      Eggs             Land O Lakes Eggs      2.0   
        3                   vanilla                       vanilla      1.0   
        4                     flour             all-purpose flour      1.5   
...                             ...                           ...      ...   
931097  10          red bell pepper               red bell pepper     0.25   
        11             Jasmine rice                  Jasmine rice      1.0   
        12                  chicken  reduced-sodium chicken broth      1.5   
        13              fresh basil                   fresh basil      1.0   
        14                     Salt        Salt and ground pepper     <NA>   

                          unit       comment preparation  \
recipe  ingredient                                         
1746116 0                 cups          <NA>    softened   
        1                 cups          <NA>        <NA>   
        2                       (yolks only)        <NA>   
        3             teaspoon          <NA>        <NA>   
        4                 cups          <NA>        <NA>   
...                        ...           ...         ...   
931097  10                cups          <NA>     chopped   
        11                 cup          <NA>        <NA>   
        12                cups          <NA>        <NA>   
        13          tablespoon          <NA>     chopped   
        14                <NA>      to taste        <NA>   

                                          ingredient_string  
recipe  ingredient                                           
1746116 0             2/3 cup Land O Lakes Butter, softened  
        1                                     1/2 cup sugar  
        2                  2 Land O Lakes Eggs (yolks only)  
        3                                1 teaspoon vanilla  
        4                      1 1/2 cups all-purpose flour  
...                                                     ...  
931097  10                  1/4 cup chopped red bell pepper  
        11                               1 cup Jasmine rice  
        12          1 1/2 cups reduced-sodium chicken broth  
        13                 1 tablespoon chopped fresh basil  
        14                  Salt and ground pepper to taste  

[2494 rows x 7 columns]

In [419]:
ingredients_df

name.name              name.description quantity  \
recipe  ingredient                                                           
1746116 0                    Butter           Land O Lakes Butter    0.667   
        1                     sugar                         sugar      0.5   
        2                      Eggs             Land O Lakes Eggs      2.0   
        3                   vanilla                       vanilla      1.0   
        4                     flour             all-purpose flour      1.5   
...                             ...                           ...      ...   
931097  10          red bell pepper               red bell pepper     0.25   
        11             Jasmine rice                  Jasmine rice      1.0   
        12                  chicken  reduced-sodium chicken broth      1.5   
        13              fresh basil                   fresh basil      1.0   
        14                     Salt        Salt and ground pepper     <NA>   

                          unit       comment preparation  \
recipe  ingredient                                         
1746116 0                 cups          <NA>    softened   
        1                 cups          <NA>        <NA>   
        2                       (yolks only)        <NA>   
        3             teaspoon          <NA>        <NA>   
        4                 cups          <NA>        <NA>   
...                        ...           ...         ...   
931097  10                cups          <NA>     chopped   
        11                 cup          <NA>        <NA>   
        12                cups          <NA>        <NA>   
        13          tablespoon          <NA>     chopped   
        14                <NA>      to taste        <NA>   

                                          ingredient_string  
recipe  ingredient                                           
1746116 0             2/3 cup Land O Lakes Butter, softened  
        1                                     1/2 cup sugar  
        2                  2 Land O Lakes Eggs (yolks only)  
        3                                1 teaspoon vanilla  
        4                      1 1/2 cups all-purpose flour  
...                                                     ...  
931097  10                  1/4 cup chopped red bell pepper  
        11                               1 cup Jasmine rice  
        12          1 1/2 cups reduced-sodium chicken broth  
        13                 1 tablespoon chopped fresh basil  
        14                  Salt and ground pepper to taste  

[2494 rows x 7 columns]

In [420]:
ingredients_df = ingredients_df.replace(r'^\s*$', pd.NA, regex=True)
ingredients_df

name.name              name.description quantity  \
recipe  ingredient                                                           
1746116 0                    Butter           Land O Lakes Butter    0.667   
        1                     sugar                         sugar      0.5   
        2                      Eggs             Land O Lakes Eggs      2.0   
        3                   vanilla                       vanilla      1.0   
        4                     flour             all-purpose flour      1.5   
...                             ...                           ...      ...   
931097  10          red bell pepper               red bell pepper     0.25   
        11             Jasmine rice                  Jasmine rice      1.0   
        12                  chicken  reduced-sodium chicken broth      1.5   
        13              fresh basil                   fresh basil      1.0   
        14                     Salt        Salt and ground pepper     <NA>   

                          unit       comment preparation  \
recipe  ingredient                                         
1746116 0                 cups          <NA>    softened   
        1                 cups          <NA>        <NA>   
        2                 <NA>  (yolks only)        <NA>   
        3             teaspoon          <NA>        <NA>   
        4                 cups          <NA>        <NA>   
...                        ...           ...         ...   
931097  10                cups          <NA>     chopped   
        11                 cup          <NA>        <NA>   
        12                cups          <NA>        <NA>   
        13          tablespoon          <NA>     chopped   
        14                <NA>      to taste        <NA>   

                                          ingredient_string  
recipe  ingredient                                           
1746116 0             2/3 cup Land O Lakes Butter, softened  
        1                                     1/2 cup sugar  
        2                  2 Land O Lakes Eggs (yolks only)  
        3                                1 teaspoon vanilla  
        4                      1 1/2 cups all-purpose flour  
...                                                     ...  
931097  10                  1/4 cup chopped red bell pepper  
        11                               1 cup Jasmine rice  
        12          1 1/2 cups reduced-sodium chicken broth  
        13                 1 tablespoon chopped fresh basil  
        14                  Salt and ground pepper to taste  

[2494 rows x 7 columns]

In [421]:
ingredients_df.iloc[2]['unit']

<NA>

## Data Types

In [422]:
ingredients_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2494 entries, (1746116, 0) to (931097, 14)
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name.name          2494 non-null   object
 1   name.description   2494 non-null   string
 2   quantity           2330 non-null   string
 3   unit               1966 non-null   string
 4   comment            280 non-null    string
 5   preparation        672 non-null    string
 6   ingredient_string  2494 non-null   string
dtypes: object(1), string(6)
memory usage: 155.6+ KB


In [423]:
ingredients_df = ingredients_df.astype({
    'name.name': 'string',
    'name.description': 'string',
    'quantity': 'Float64',
    'unit': 'string',
    'comment': 'string',
    'preparation': 'string',
    'ingredient_string': 'string'
})

In [424]:
ingredients_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2494 entries, (1746116, 0) to (931097, 14)
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name.name          2494 non-null   string 
 1   name.description   2494 non-null   string 
 2   quantity           2330 non-null   Float64
 3   unit               1966 non-null   string 
 4   comment            280 non-null    string 
 5   preparation        672 non-null    string 
 6   ingredient_string  2494 non-null   string 
dtypes: Float64(1), string(6)
memory usage: 158.0 KB


## Unit Tagging

The above will cover all that is required to find ingredients by their name in the `food_df`. From this finding thier density requires another step: to find the appropriate portion measure / size in the `food_portion_df`. To best do this we can homogenise the units, which is easily done as they are not so expansive, they are only a list of ~30. Again we can do this with NLP.

In [425]:
ingredients_df['unit'].value_counts().head(20)

unit
cups           545
cup            166
teaspoons      146
tablespoons    106
tsps            94
can             81
teaspoon        71
Tbsps           61
tablespoon      61
tsp             53
lbs             40
ounces          40
large           34
cloves          30
lb              27
medium          27
Tbsp            27
oz              22
cans            20
tbsps           19
Name: count, dtype: Int64

In [426]:
ingredients_df['unit_tags'], ingredients_df['unit_remainders'], ingredients_df['unit_type'] = zip(*ingredients_df['unit'].apply(clean_ingredient_string).progress_apply(tag_units))
ingredients_df[['unit', 'unit_tags', 'unit_remainders', 'unit_type']]

100%|██████████| 2494/2494 [00:03<00:00, 831.24it/s]


unit     unit_tags unit_remainders unit_type
recipe  ingredient                                                    
1746116 0                 cups         [cup]              []    volume
        1                 cups         [cup]              []    volume
        2                 <NA>            []              []   portion
        3             teaspoon    [teaspoon]              []    volume
        4                 cups         [cup]              []    volume
...                        ...           ...             ...       ...
931097  10                cups         [cup]              []    volume
        11                 cup         [cup]              []    volume
        12                cups         [cup]              []    volume
        13          tablespoon  [tablespoon]              []    volume
        14                <NA>            []              []   portion

[2494 rows x 4 columns]

In [427]:
ingredients_df[(ingredients_df['unit_tags'].str.len() == 0) & (ingredients_df['unit'] != '')]

name.name name.description  quantity  \
recipe  ingredient                                               
1828339 8           ground cinnamon  ground cinnamon       1.0   
        9           ground allspice  ground allspice       1.0   
1703    3                 mincemeat        mincemeat       1.0   
382666  1                    butter           butter       1.5   
2195357 0             blue-eyed cod    blue-eyed cod       4.0   
...                             ...              ...       ...   
2177242 3                  scallion         scallion       3.0   
64662   0                 margarine        margarine       1.0   
1936429 1                    garlic           garlic       2.0   
1401651 4             ras el hanout    ras el hanout      <NA>   
1357213 2                    butter           butter       2.0   

                              unit          comment       preparation  \
recipe  ingredient                                                      
1828339 8                    pinch             <NA>              <NA>   
        9                    pinch             <NA>              <NA>   
1703    3                      jar             <NA>              <NA>   
382666  1                   sticks             <NA>              <NA>   
2195357 0                  Cutlets             <NA>              <NA>   
...                            ...              ...               ...   
2177242 3                    heads  white part only    finely chopped   
64662   0                    stick             <NA>          softened   
1936429 1                   cloves             <NA>  coarsely chopped   
1401651 4           generous pinch             <NA>              <NA>   
1357213 2                   sticks             <NA>              <NA>   

                                                   ingredient_string  \
recipe  ingredient                                                     
1828339 8                                    1 pinch ground cinnamon   
        9                                    1 pinch ground allspice   
1703    3                                            1 jar mincemeat   
382666  1                                        1 1/2 sticks butter   
2195357 0                                  4 x Cutlets blue-eyed cod   
...                                                              ...   
2177242 3           3 scallion heads white part only, finely chopped   
64662   0                                1 stick margarine, softened   
1936429 1                          2 cloves garlic, coarsely chopped   
1401651 4                             generous pinches ras el hanout   
1357213 2                                            2 sticks butter   

                   unit_tags    unit_remainders unit_type  
recipe  ingredient                                         
1828339 8                 []            [pinch]   portion  
        9                 []            [pinch]   portion  
1703    3                 []              [jar]   portion  
382666  1                 []            [stick]   portion  
2195357 0                 []           [cutlet]   portion  
...                      ...                ...       ...  
2177242 3                 []             [head]   portion  
64662   0                 []            [stick]   portion  
1936429 1                 []            [clove]   portion  
1401651 4                 []  [generous, pinch]   portion  
1357213 2                 []            [stick]   portion  

[154 rows x 10 columns]

In [428]:
ingredients_df['unit_type'].value_counts()

unit_type
volume     1497
portion     815
weight      182
Name: count, dtype: int64

In [429]:
ingredients_df['unit_type'] = ingredients_df['unit_type'].astype('category')

# Creating Expanded Ingredients DF

What we want here is to set the dataframe up for use in other sections/modules. All of these steps should go here.

This includes:
- removing unnecessary columns
- cleaning data
- homogenising data as above

Here we do any dataframe operations required for the joining. This is done to the full dataframe at the beginning, to have them all ready for the join operation, preventing the computations being done numerous times.

## Extracting Search Terms

The next chapters will involve joining various dataframes with this one ie. matching these ingredients with those of other dataframes through searching through them in a methodical manner. To do this we have devised a function that takes in a list of **orderd** search terms, using these to refine the search towards the best match. This can be arranged as a set of columns for each ingredient.

Emphasis is put onto the ordering of these search terms, as we want to carefully lead our search to the match, avoiding getting stuck in incorrect searches. The rest of this section will do this by ordering the columns, and extracting their information in a methodical way.

### Selecting Columns

#decision

Here decisions need to be made in selecting which part of the ingredient info we need for searching. We want to get a specific match on an ingredient, but again we must make sure to avoid 'over-searching' and matching with an obscure ingredient.

We definitely don't want to include information about how an ingredient should be prepared, do we want comments? Comments seem to cause more harm than good in the search, so these are removed.

In [430]:
ingredients_df.shape

(2494, 10)

#### Nouns Separation

When it comes to matching the ingredient string, nouns are the most important search term. Adjectives matches are more optional, and sometimes hinder results by matching with oddly specific matches that are too detailed and stem away from the base ingredient. ie. chopped onion matching with "chicken breast with chopped onion and chopped pepper." instead of simply "onion".

In [431]:
nltk.pos_tag(mt.tokenize('red bell pepper'))

[('red', 'JJ'), ('bell', 'NN'), ('pepper', 'IN')]

One more point of note, the tag of a word depends on its position relative to other words. This can cause issues when stripping words away.

In [432]:
assert tag_ingredient_string('chopped onions') != [*tag_ingredient_string('chopped'), *tag_ingredient_string('onion')]
tag_ingredient_string('chopped onions'), [*tag_ingredient_string('chopped'), *tag_ingredient_string('onion')]

([{'word': 'chopped', 'tag': 'adj'}, {'word': 'onions', 'tag': 'noun'}],
 [{'word': 'chopped', 'tag': 'noun'}, {'word': 'onion', 'tag': 'noun'}])

It would be best if we instead tagged the list before this was split.

Here we want to perform it on `name.full`, then find the indices at which the `name.name` and `name.description` was split, use these to find the tags of the string at that index, and match that to the original tags.

In [433]:
#| export
def tokenize_with_spans(txt):
    tokens=mt.tokenize(txt)
    offset = 0
    for token in tokens:
        offset = txt.find(token, offset)
        yield token, offset, offset+len(token)
        offset += len(token)

In [434]:
list(tokenize_with_spans('red bell pepper'))

[('red', 0, 3), ('bell', 4, 8), ('pepper', 9, 15)]

In [435]:
ingredient = ingredients_df.loc[58195, 4]
ingredient

name.name                      butter
name.description           buttermilk
quantity                          1.0
unit                              cup
comment                          <NA>
preparation                      <NA>
ingredient_string    1 cup buttermilk
unit_tags                       [cup]
unit_remainders                    []
unit_type                      volume
Name: (58195, 4), dtype: object

In [436]:
#| export
def get_match_idxs(ingredient_string, search_string):
    idx = ingredient_string.find(search_string)
    if idx == -1: 
        return None
    return set(range(idx, idx+len(search_string)))

In [437]:
assert get_match_idxs('1/2 cup sugar', 'sugar') == set(range(8,13))
assert get_match_idxs('8 ounces blueberries', 'blueberry') == None

In [438]:
#| export
def split_nouns(ingredient_string, cut_string):

    if not cut_string: return ([], [])

    # getting tags of full string
    tokens_spanned = list(tokenize_with_spans(ingredient_string))
    tokens_spanned_tags = nltk.pos_tag(list(zip(*tokens_spanned))[0])
    if tokens_spanned_tags: tokens_spanned_tags[-1] = (tokens_spanned_tags[-1][0], 'NN') # setting last word as noun (as this is describing item)
    tokens_spanned_tagged = [word[0] + word[1] for word in list(zip(tokens_spanned, tokens_spanned_tags))] # joining the two together
    tokens_spanned_tagged = list(zip(*[t for i, t in enumerate(list(zip(*tokens_spanned_tagged))) if i != 3])) # removing 3rd index of each tuple in list
    
    # using full string tags to tag cut string
    match_idxs = get_match_idxs(ingredient_string, cut_string)
    if not match_idxs: return pd.NA, pd.NA
    tags = [t for t in tokens_spanned_tagged if match_idxs.intersection(range(*t[1:3]))]
    
    # splitting nouns
    tags_split = [t[0] for t in tags if t[3].startswith('NN')], [t[0] for t in tags if not t[3].startswith('NN')]

    # limit to 6 words for each split
    tags_split = tuple([tags[0:6] for tags in tags_split])

    return tags_split

In [439]:
assert split_nouns('1/2 cup granulated sugar', 'granulated sugar') == (['sugar'], ['granulated'])
assert split_nouns('8 ounces blueberries', 'blueberry') == (pd.NA, pd.NA)
assert split_nouns('8 ounces blueberries', '') == ([], [])
assert split_nouns('red bell pepper', 'red bell pepper') == (['bell','pepper'], ['red']) # final word should be noun always

In [440]:
#| export
def split_ingredient_fields_by_noun(ingredient, debug=False):
    for col in ingredient.index:
        if col == 'ingredient_string': continue
        ingredient[col + '.nouns'], ingredient[col + '.others'] = split_nouns(ingredient['name.description'], ingredient[col])
        if debug:
            if ingredient[col] != '' and (len(ingredient[col + '.nouns']) + len(ingredient[col + '.others']) == 0):
                print('WARN: Missing ingredient tags', ingredient.name, col, ingredient[col], ingredient['ingredient_string'], sep=' | ')
    return ingredient

In [441]:
expanded_ingredients_df = ingredients_df[['name.name', 'name.description', 'ingredient_string']]
expanded_ingredients_df = expanded_ingredients_df.apply(split_ingredient_fields_by_noun, axis=1)

expanded_ingredients_df.drop([col for col in expanded_ingredients_df.columns if not any(s in col for s in ['nouns', 'others'])], axis=1, inplace=True)
expanded_ingredients_df.head()

name.name.nouns name.name.others    name.description.nouns  \
recipe  ingredient                                                              
1746116 0                 [Butter]               []  [Land, O, Lakes, Butter]   
        1                  [sugar]               []                   [sugar]   
        2                   [Eggs]               []    [Land, O, Lakes, Eggs]   
        3                [vanilla]               []                 [vanilla]   
        4                  [flour]               []                   [flour]   

                   name.description.others  
recipe  ingredient                          
1746116 0                               []  
        1                               []  
        2                               []  
        3                               []  
        4                    [all-purpose]

In [442]:
ingredients_df.loc[767911, 1]

name.name                          onion
name.description                   onion
quantity                             1.0
unit                                 cup
comment                             <NA>
preparation                      chopped
ingredient_string    1 cup chopped onion
unit_tags                          [cup]
unit_remainders                       []
unit_type                         volume
Name: (767911, 1), dtype: object

In [443]:
# split expand lists into individual columns
original_cols = expanded_ingredients_df.columns
for expand_col in original_cols:
    expanded = pd.DataFrame(expanded_ingredients_df[expand_col].tolist(), index=expanded_ingredients_df.index)
    expanded.columns = [expand_col + '.' + str(c) for c in expanded.columns]
    expanded_ingredients_df = expanded_ingredients_df.join(expanded)
expanded_ingredients_df.drop(columns=original_cols, inplace=True)
expanded_ingredients_df.columns

Index(['name.name.nouns.0', 'name.name.nouns.1', 'name.name.nouns.2',
       'name.name.nouns.3', 'name.name.nouns.4', 'name.name.others.0',
       'name.name.others.1', 'name.name.others.2', 'name.name.others.3',
       'name.description.nouns.0', 'name.description.nouns.1',
       'name.description.nouns.2', 'name.description.nouns.3',
       'name.description.nouns.4', 'name.description.nouns.5',
       'name.description.others.0', 'name.description.others.1',
       'name.description.others.2', 'name.description.others.3',
       'name.description.others.4', 'name.description.others.5'],
      dtype='object')

In [444]:
expanded_ingredients_df = expanded_ingredients_df.convert_dtypes()
print(list(expanded_ingredients_df.dtypes))

[string[python], string[python], string[python], string[python], string[python], string[python], string[python], string[python], string[python], string[python], string[python], string[python], string[python], string[python], string[python], string[python], string[python], string[python], string[python], string[python], string[python]]


## Reversing Column Order

When dealing with nouns, most of the time gramatically the most important and core noun comes last. eg. 'apple cider vinegar' where 'vinegar' is the core ingredient.

In [445]:
split_types = ['nouns', 'others']
original_cols = list(dict.fromkeys([re.sub(r'\.(nouns|others).*', '', c) for c in expanded_ingredients_df.columns]))
reordered_cols = []
for col in original_cols:
    for word_type in split_types:
        col_expanded = [c for c in expanded_ingredients_df.columns if col in c and word_type in c]
        if word_type =='nouns': col_expanded.reverse()
        reordered_cols.extend(col_expanded)

reordered_cols
    
expanded_ingredients_df = expanded_ingredients_df[reordered_cols]
expanded_ingredients_df.columns

Index(['name.name.nouns.4', 'name.name.nouns.3', 'name.name.nouns.2',
       'name.name.nouns.1', 'name.name.nouns.0', 'name.name.others.0',
       'name.name.others.1', 'name.name.others.2', 'name.name.others.3',
       'name.description.nouns.5', 'name.description.nouns.4',
       'name.description.nouns.3', 'name.description.nouns.2',
       'name.description.nouns.1', 'name.description.nouns.0',
       'name.description.others.0', 'name.description.others.1',
       'name.description.others.2', 'name.description.others.3',
       'name.description.others.4', 'name.description.others.5'],
      dtype='object')

In [446]:
assert int(expanded_ingredients_df.columns[0].split('.')[-1]) > 0

For more accurate grammar tagging, we preferred the description to contain the full string. To make sure we're not searching for the same search term twice we can now remove them here ready for exporting.

In [447]:
#| export
def remove_name_from_description(ingredient):
    ingredient_cols = ingredient[ingredient.notnull()].index
    ingredient_name_cols = ingredient_cols[ingredient_cols.str.startswith('name.name')]
    ingredient_desc_cols = ingredient_cols[ingredient_cols.str.startswith('name.description')]
    matching_cols = ingredient[ingredient_desc_cols][ingredient[ingredient_desc_cols].apply(lambda x: x in ingredient[ingredient_name_cols].values)].index
    ingredient[matching_cols] = pd.NA
    return ingredient

In [448]:
expanded_ingredient = expanded_ingredients_df.iloc[0]
expanded_ingredient

name.name.nouns.4              <NA>
name.name.nouns.3              <NA>
name.name.nouns.2              <NA>
name.name.nouns.1              <NA>
name.name.nouns.0            Butter
name.name.others.0             <NA>
name.name.others.1             <NA>
name.name.others.2             <NA>
name.name.others.3             <NA>
name.description.nouns.5       <NA>
name.description.nouns.4       <NA>
name.description.nouns.3     Butter
name.description.nouns.2      Lakes
name.description.nouns.1          O
name.description.nouns.0       Land
name.description.others.0      <NA>
name.description.others.1      <NA>
name.description.others.2      <NA>
name.description.others.3      <NA>
name.description.others.4      <NA>
name.description.others.5      <NA>
Name: (1746116, 0), dtype: string

In [449]:
assert pd.isna(remove_name_from_description(expanded_ingredient)['name.description.nouns.3'])

In [450]:
expanded_ingredients_df = expanded_ingredients_df.apply(remove_name_from_description, axis=1)

In [451]:
assert pd.isna(expanded_ingredients_df.iloc[0])['name.description.nouns.3']

# Processsing (2)

Processing stages to be applied after the `expanded_ingredients_df` has been created.

## Cleaning Columns

Note this was saved until after separating nouns/others in case it altered phrases.

In [452]:
ingredients_df[['name.name', 'name.description']] = ingredients_df[['name.name', 'name.description']].map(clean_ingredient_string)
ingredients_df.head()

name.name   name.description  quantity      unit  \
recipe  ingredient                                                    
1746116 0             butter   land lake butter     0.667      cups   
        1              sugar              sugar       0.5      cups   
        2                egg      land lake egg       2.0      <NA>   
        3            vanilla            vanilla       1.0  teaspoon   
        4              flour  all-purpose flour       1.5      cups   

                         comment preparation  \
recipe  ingredient                             
1746116 0                   <NA>    softened   
        1                   <NA>        <NA>   
        2           (yolks only)        <NA>   
        3                   <NA>        <NA>   
        4                   <NA>        <NA>   

                                        ingredient_string   unit_tags  \
recipe  ingredient                                                      
1746116 0           2/3 cup Land O Lakes Butter, softened       [cup]   
        1                                   1/2 cup sugar       [cup]   
        2                2 Land O Lakes Eggs (yolks only)          []   
        3                              1 teaspoon vanilla  [teaspoon]   
        4                    1 1/2 cups all-purpose flour       [cup]   

                   unit_remainders unit_type  
recipe  ingredient                            
1746116 0                       []    volume  
        1                       []    volume  
        2                       []   portion  
        3                       []    volume  
        4                       []    volume

In [453]:
expanded_ingredients_df = expanded_ingredients_df.map(clean_ingredient_string)
expanded_ingredients_df = expanded_ingredients_df.replace(r'^\s*$', pd.NA, regex=True)
expanded_ingredients_df.head()

name.name.nouns.4 name.name.nouns.3 name.name.nouns.2  \
recipe  ingredient                                                         
1746116 0                       <NA>              <NA>              <NA>   
        1                       <NA>              <NA>              <NA>   
        2                       <NA>              <NA>              <NA>   
        3                       <NA>              <NA>              <NA>   
        4                       <NA>              <NA>              <NA>   

                   name.name.nouns.1 name.name.nouns.0 name.name.others.0  \
recipe  ingredient                                                          
1746116 0                       <NA>            butter               <NA>   
        1                       <NA>             sugar               <NA>   
        2                       <NA>               egg               <NA>   
        3                       <NA>           vanilla               <NA>   
        4                       <NA>             flour               <NA>   

                   name.name.others.1 name.name.others.2 name.name.others.3  \
recipe  ingredient                                                            
1746116 0                        <NA>               <NA>               <NA>   
        1                        <NA>               <NA>               <NA>   
        2                        <NA>               <NA>               <NA>   
        3                        <NA>               <NA>               <NA>   
        4                        <NA>               <NA>               <NA>   

                   name.description.nouns.5  ... name.description.nouns.3  \
recipe  ingredient                           ...                            
1746116 0                              <NA>  ...                     <NA>   
        1                              <NA>  ...                     <NA>   
        2                              <NA>  ...                     <NA>   
        3                              <NA>  ...                     <NA>   
        4                              <NA>  ...                     <NA>   

                   name.description.nouns.2 name.description.nouns.1  \
recipe  ingredient                                                     
1746116 0                              lake                     <NA>   
        1                              <NA>                     <NA>   
        2                              lake                     <NA>   
        3                              <NA>                     <NA>   
        4                              <NA>                     <NA>   

                   name.description.nouns.0 name.description.others.0  \
recipe  ingredient                                                      
1746116 0                              land                      <NA>   
        1                              <NA>                      <NA>   
        2                              land                      <NA>   
        3                              <NA>                      <NA>   
        4                              <NA>               all-purpose   

                   name.description.others.1 name.description.others.2  \
recipe  ingredient                                                       
1746116 0                               <NA>                      <NA>   
        1                               <NA>                      <NA>   
        2                               <NA>                      <NA>   
        3                               <NA>                      <NA>   
        4                               <NA>                      <NA>   

                   name.description.others.3 name.description.others.4  \
recipe  ingredient                                                       
1746116 0                               <NA>                      <NA>   
        1                               <NA>                      <NA>   
        2           

In [454]:
expanded_ingredients_df.head()

name.name.nouns.4 name.name.nouns.3 name.name.nouns.2  \
recipe  ingredient                                                         
1746116 0                       <NA>              <NA>              <NA>   
        1                       <NA>              <NA>              <NA>   
        2                       <NA>              <NA>              <NA>   
        3                       <NA>              <NA>              <NA>   
        4                       <NA>              <NA>              <NA>   

                   name.name.nouns.1 name.name.nouns.0 name.name.others.0  \
recipe  ingredient                                                          
1746116 0                       <NA>            butter               <NA>   
        1                       <NA>             sugar               <NA>   
        2                       <NA>               egg               <NA>   
        3                       <NA>           vanilla               <NA>   
        4                       <NA>             flour               <NA>   

                   name.name.others.1 name.name.others.2 name.name.others.3  \
recipe  ingredient                                                            
1746116 0                        <NA>               <NA>               <NA>   
        1                        <NA>               <NA>               <NA>   
        2                        <NA>               <NA>               <NA>   
        3                        <NA>               <NA>               <NA>   
        4                        <NA>               <NA>               <NA>   

                   name.description.nouns.5  ... name.description.nouns.3  \
recipe  ingredient                           ...                            
1746116 0                              <NA>  ...                     <NA>   
        1                              <NA>  ...                     <NA>   
        2                              <NA>  ...                     <NA>   
        3                              <NA>  ...                     <NA>   
        4                              <NA>  ...                     <NA>   

                   name.description.nouns.2 name.description.nouns.1  \
recipe  ingredient                                                     
1746116 0                              lake                     <NA>   
        1                              <NA>                     <NA>   
        2                              lake                     <NA>   
        3                              <NA>                     <NA>   
        4                              <NA>                     <NA>   

                   name.description.nouns.0 name.description.others.0  \
recipe  ingredient                                                      
1746116 0                              land                      <NA>   
        1                              <NA>                      <NA>   
        2                              land                      <NA>   
        3                              <NA>                      <NA>   
        4                              <NA>               all-purpose   

                   name.description.others.1 name.description.others.2  \
recipe  ingredient                                                       
1746116 0                               <NA>                      <NA>   
        1                               <NA>                      <NA>   
        2                               <NA>                      <NA>   
        3                               <NA>                      <NA>   
        4                               <NA>                      <NA>   

                   name.description.others.3 name.description.others.4  \
recipe  ingredient                                                       
1746116 0                               <NA>                      <NA>   
        1                               <NA>                      <NA>   
        2           

## Filtering Strings

We want to do a bit of filtering here, mainly with 'seasoning' in mind. This is an unnecessary addition in most recipes, with an exception being in baking where this matters. If it does matter it will have a specified quantity and unit. 

In [455]:
ingredient = expanded_ingredients_df.loc[2058895, 6]
ingredient[ingredient.notnull()]

name.name.nouns.0             salt
name.description.nouns.1    pepper
Name: (2058895, 6), dtype: object

In [456]:
#| export
def filter_patterns(ingredient, filters):

    cols = ingredient.index[ingredient.notnull()]

    for filter_words in filters:
        if all([any([filter_word == ingredient[col] for col in cols]) for filter_word in filter_words]):
            return True
        
    return False

filters = [['salt'],['ground', 'pepper'],['black', 'pepper'], ['pepper']]

In [457]:
assert filter_patterns(expanded_ingredients_df.loc[2058895, 6], filters) == True

In [458]:
# only filtering the ingredients which do not specify their unit & quantity - otherwise we are able to assess.
quantity_filter = (ingredients_df['unit'] == '') & (ingredients_df['quantity'] == '')

word_filter = expanded_ingredients_df.apply(filter_patterns, args=(filters,), axis=1)

print((quantity_filter & word_filter).sum(), expanded_ingredients_df.shape[0])

expanded_ingredients_df = expanded_ingredients_df[~(quantity_filter & word_filter)]

ingredients_df = ingredients_df.loc[expanded_ingredients_df.index]

0 2494


# Saving

In [459]:
ingredients_df

name.name              name.description  quantity  \
recipe  ingredient                                                            
1746116 0                    butter              land lake butter     0.667   
        1                     sugar                         sugar       0.5   
        2                       egg                 land lake egg       2.0   
        3                   vanilla                       vanilla       1.0   
        4                     flour             all-purpose flour       1.5   
...                             ...                           ...       ...   
931097  9                 red onion                     red onion      0.25   
        10          red bell pepper               red bell pepper      0.25   
        11             jasmine rice                  jasmine rice       1.0   
        12                  chicken  reduced-sodium chicken broth       1.5   
        13              fresh basil                   fresh basil       1.0   

                          unit       comment     preparation  \
recipe  ingredient                                             
1746116 0                 cups          <NA>        softened   
        1                 cups          <NA>            <NA>   
        2                 <NA>  (yolks only)            <NA>   
        3             teaspoon          <NA>            <NA>   
        4                 cups          <NA>            <NA>   
...                        ...           ...             ...   
931097  9                 cups          <NA>  finely chopped   
        10                cups          <NA>         chopped   
        11                 cup          <NA>            <NA>   
        12                cups          <NA>            <NA>   
        13          tablespoon          <NA>         chopped   

                                          ingredient_string     unit_tags  \
recipe  ingredient                                                          
1746116 0             2/3 cup Land O Lakes Butter, softened         [cup]   
        1                                     1/2 cup sugar         [cup]   
        2                  2 Land O Lakes Eggs (yolks only)            []   
        3                                1 teaspoon vanilla    [teaspoon]   
        4                      1 1/2 cups all-purpose flour         [cup]   
...                                                     ...           ...   
931097  9                 1/4 cup red onion, finely chopped         [cup]   
        10                  1/4 cup chopped red bell pepper         [cup]   
        11                               1 cup Jasmine rice         [cup]   
        12          1 1/2 cups reduced-sodium chicken broth         [cup]   
        13                 1 tablespoon chopped fresh basil  [tablespoon]   

                   unit_remainders unit_type  
recipe  ingredient                            
1746116 0                       []    volume  
        1                       []    volume  
        2                       []   portion  
        3                       []    volume  
        4                       []    volume  
...                            ...       ...  
931097  9                       []    volume  
        10                      []    volume  
        11                      []    volume  
        12                      []    volume  
        13                      []    volume  

[2450 rows x 10 columns]

In [460]:
ingredients_df.loc[2006319]

,name.name,name.description,quantity,unit,comment,preparation,ingredient_string,unit_tags,unit_remainders,unit_type
ingredient,,,,,,,,,,
0,watermelon,watermelon,1.0,<NA>,<NA>,<NA>,1 watermelon,[],[],portion
1,cantaloupe,cantaloupe,1.0,<NA>,<NA>,<NA>,1 cantaloupe,[],[],portion
2,raspberry,raspberry,8.0,ounces,<NA>,<NA>,8 ounces raspberries,[ounce],[],weight
3,blackberry,blackberry,8.0,ounces,<NA>,<NA>,8 ounces blackberries,[ounce],[],weight
4,blueberry,blueberry,8.0,ounces,<NA>,<NA>,8 ounces blueberries,[ounce],[],weight


In [461]:
ingredients_df.to_feather('../data/local/recipe/partial/ingredients/0.feather')
expanded_ingredients_df.to_feather('../data/local/recipe/partial/expanded_ingredients/0.feather')

In [463]:
from nbdev import nbdev_export; nbdev_export()